In [ ]:
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, Activation
import matplotlib.pyplot as plt
import numpy as np
import keras
from sklearn import metrics

This notebook contains a simple image classification example using a small convolutional neural network, the Keras deep learning package, and the Fashion-MNIST data set.

## Load data

The Fashion-MNIST data set can be loaded directly from Keras.

In [ ]:
(x_train_orig, y_train_orig), (x_test_orig, y_test_orig) = fashion_mnist.load_data()

In [ ]:
def show_img(img, ax):
    ax.imshow(img, cmap='Greys')
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.set_xticks([])
    ax.set_yticks([])
    ax.axis('off')

Fashion-MNIST contains 28x28 greyscale images of cloting articles belonging to 10 categories. The data set contains 60,000 training and 10,000 test images. The data set was created by Zalandor Research as a drop-in replacement for the popular MNIST dataset of hand-written digits.

In [ ]:
row_count = 2
column_count = 10

fig, ax = plt.subplots(row_count, column_count, figsize=(column_count, row_count))

for x in range(row_count):
    for y in range(column_count):
        show_img(x_train_orig[np.random.randint(0, x_train_orig.shape[0] - 1)], ax[x, y])

plt.show()

## Pre-prcess data

In [ ]:
img_rows = 28
img_cols = 28
num_classes = 10

Convert the images to the format expected by Keras.

In [ ]:
x_train = x_train_orig.reshape(x_train_orig.shape[0], img_rows, img_cols, 1)
x_test = x_test_orig.reshape(x_test_orig.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

y_train = keras.utils.to_categorical(y_train_orig, num_classes)
y_test = keras.utils.to_categorical(y_test_orig, num_classes)

In order to speed up training, 5% or the training and 10% of the test images are used.

In [ ]:
mask_train = np.random.binomial(1, .05, y_train.shape[0]).astype('bool')
mask_test = np.random.binomial(1, .1, y_test.shape[0]).astype('bool')

x_train = x_train[mask_train]
x_test = x_test[mask_test]

y_train = y_train[mask_train]
y_test = y_test[mask_test]

print('Number of training images: {}'.format(x_train.shape[0]))
print('Number of test images:     {}'.format(x_test.shape[0]))

## Define model

We use a simple convolutional neural network consisting of three convolutional layers with progressively smaller image size and increasing depth and one fully connected layer. The softmax activation ensures, that the 10 output values, interpreted as the probability, that the image belongs to that category, are all between 0.0 and 1.0, and their sum is always 1.0

In [ ]:
model = Sequential();
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=input_shape))
model.add(Conv2D(64, (3, 3), strides=2, padding='same', activation='relu'))
model.add(Conv2D(128, (3, 3), strides=2, padding='same', activation='relu'))
model.add(Flatten())
model.add(Dense(10))
model.add(Activation('softmax'))

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

## Train model

In [ ]:
batch_size = 32
epochs = 10

We train the network over 10 epochs

In [ ]:
%%time
train_hist = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test))

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 5))

# Loss Curves
ax[0].plot(train_hist.history['loss'],'r',linewidth=3.0)
ax[0].plot(train_hist.history['val_loss'],'b',linewidth=3.0)
ax[0].grid(True, axis = 'y')
ax[0].legend(['Training loss', 'Validation Loss'])
ax[0].set_ylim([0, 1])
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Loss')
ax[0].set_title('Loss Curves')

# Accuracy Curves
ax[1].plot(train_hist.history['categorical_accuracy'],'r',linewidth=3.0)
ax[1].plot(train_hist.history['val_categorical_accuracy'],'b',linewidth=3.0)
ax[1].grid(True, axis = 'y')
ax[1].legend(['Training Accuracy', 'Validation Accuracy'])
ax[1].set_ylim([0, 1])
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Accuracy')
ax[1].set_title('Accuracy Curves')

plt.show()

The development of the loss function (the value that the training tries to minimize) and the accuracy during the training.

## Validate

In [ ]:
y_test_pred = model.predict(x_test)

y_test_value = np.argmax(y_test, axis=1)
y_test_pred_value = np.argmax(y_test_pred, axis=1)

In [ ]:
confusion_matrix = metrics.confusion_matrix(y_test_value, y_test_pred_value)
plt.imshow(np.log(confusion_matrix), cmap='Blues')
plt.xticks([], [])
plt.yticks([], [])
plt.show()

The confusion matrix shows the distribution of the observations in terms of true category (rows) and predicted categories (columns). The observations on the diagonal are correctly classified.

In [ ]:
print('Accuracy:', metrics.accuracy_score(y_test_value, y_test_pred_value))